---
# <U>***Importing librabries***

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from dateparser import parse
import warnings
import dateparser


---
* ***Creating SanFran data frame for the past 3 years to current.***

In [23]:
df = pd.read_excel("nfl.xlsx")
df.set_index("Date", inplace=True)
df_years = df.loc["2017":"2020"]
df_san_fran = df_years.loc[(df_years["Home Team"] == "df_San Francisco 49ers") | (df_years["Away Team"] == "San Francisco 49ers")]
df_san_fran = df_san_fran[["Home Team", "Away Team", "Home Score", "Away Score", "Total Score Open"]]
df_san_fran["Home Team"].loc["2019-10-20"] = "Washington Football Team"
df_san_fran["Home Team"].loc["2017-10-15"] = "Washington Football Team"
df_san_fran.tail()

,Home Team,Away Team,Home Score,Away Score,Total Score Open
Date,,,,,
2017-10-29,Philadelphia Eagles,San Francisco 49ers,33,10,47.5
2017-10-15,Washington Football Team,San Francisco 49ers,26,24,46.5
2017-10-08,Indianapolis Colts,San Francisco 49ers,26,23,43.0
2017-10-01,Arizona Cardinals,San Francisco 49ers,18,15,44.5
2017-09-17,Seattle Seahawks,San Francisco 49ers,12,9,43.0


In [24]:
df_san_fran.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27 entries, 2020-09-27 to 2017-09-17
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Home Team         27 non-null     object 
 1   Away Team         27 non-null     object 
 2   Home Score        27 non-null     int64  
 3   Away Score        27 non-null     int64  
 4   Total Score Open  27 non-null     float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.5+ KB


---
* ***Creating data frame with the days, and time of day that they played.***

In [4]:
def custom_date(my_row, year):
    d = str(my_row) + f", {year}"
    return d

In [5]:
years = ['2017', '2018', '2019','2020']
list_of_dfs = []
for i in years:
    list_of_dfs.append(pd.read_html(f'https://www.pro-football-reference.com/teams/sfo/{i}.htm'))

In [6]:
yr_2017 = list_of_dfs[0][1]
yr_2018 = list_of_dfs[1][1]
yr_2019 = list_of_dfs[2][1]
yr_2020 = list_of_dfs[3][1]

In [7]:
df_2017 = yr_2017[["Unnamed: 3_level_0","Unnamed: 1_level_0", "Unnamed: 2_level_0"]]
df_2018 = yr_2017[["Unnamed: 3_level_0","Unnamed: 1_level_0", "Unnamed: 2_level_0"]]
df_2019 = yr_2017[["Unnamed: 3_level_0","Unnamed: 1_level_0", "Unnamed: 2_level_0"]]
df_2020 = yr_2017[["Unnamed: 3_level_0","Unnamed: 1_level_0", "Unnamed: 2_level_0"]]

In [8]:
df_2017.columns = ["Time of Game", "Game Day", "Date"]
df_2018.columns = ["Time of Game", "Game Day", "Date"]
df_2019.columns = ["Time of Game", "Game Day", "Date"]
df_2020.columns = ["Time of Game", "Game Day", "Date"]

In [9]:
df_2017['dt'] = df_2017['Date'].apply(lambda row: custom_date(row, '2017'))
df_2018['dt'] = df_2018['Date'].apply(lambda row: custom_date(row, '2018'))
df_2019['dt'] = df_2019['Date'].apply(lambda row: custom_date(row, '2019'))
df_2020['dt'] = df_2020['Date'].apply(lambda row: custom_date(row, '2020'))

In [10]:
df_2017['clean_dt'] = df_2017['dt'].apply(lambda row: dateparser.parse(row).strftime('%Y-%m-%d'))
df_2018['clean_dt'] = df_2018['dt'].apply(lambda row: dateparser.parse(row).strftime('%Y-%m-%d'))
df_2019['clean_dt'] = df_2019['dt'].apply(lambda row: dateparser.parse(row).strftime('%Y-%m-%d'))
df_2020['clean_dt'] = df_2020['dt'].apply(lambda row: dateparser.parse(row).strftime('%Y-%m-%d'))

In [11]:
df_2017 = df_2017.drop(columns=["dt", "Date"])
df_2018 = df_2018.drop(columns=["dt", "Date"])
df_2019 = df_2019.drop(columns=["dt", "Date"])
df_2020 = df_2020.drop(columns=["dt", "Date"])

In [12]:
df_2017.rename(columns = {'clean_dt':'Date'}, inplace = True)
df_2018.rename(columns = {'clean_dt':'Date'}, inplace = True) 
df_2019.rename(columns = {'clean_dt':'Date'}, inplace = True) 
df_2020.rename(columns = {'clean_dt':'Date'}, inplace = True) 

In [13]:
df_2017.set_index("Date", inplace=True)
df_2018.set_index("Date", inplace=True)
df_2019.set_index("Date", inplace=True)
df_2020.set_index("Date", inplace=True)

In [14]:
df_day_time = pd.concat([df_2017,df_2018,df_2019,df_2020])
df_day_time.tail()

,Time of Game,Game Day
Date,,
2020-12-03,1:00PM ET,Sun
2020-12-10,1:00PM ET,Sun
2020-12-17,4:25PM ET,Sun
2020-12-24,4:05PM ET,Sun
2020-12-31,4:25PM ET,Sun


In [15]:
#df_time_day = pd.read_html("https://www.sbnation.com/2017/4/20/15376922/2017-nfl-schedule-released-dates-times-highlights")
#df_time_day = df_time_day[0]
#df_time_day = df_time_day.drop(columns=(["Date", "Away Team", "Week", "TV"]))
#df_time_day.tail()

---
* ***Combing, and cleaning, data frames***

In [16]:
df_stadium = pd.read_html("https://en.wikipedia.org/wiki/List_of_current_National_Football_League_stadiums")
df_stadium = df_stadium[1]
df_stadium = df_stadium.drop(columns=(["Image", "Capacity", "Opened", "Ref(s)"]))
df_stadium = df_stadium.append(df_stadium.iloc[[20,25]])
df_stadium.reset_index(inplace=True,drop=True)
# Chaning the names of a few rows
df_stadium["Team(s)"].loc[20] = "New York Giants"
df_stadium["Team(s)"].loc[30] = "New York Jets"
df_stadium["Team(s)"].loc[25] = "Los Angeles Rams"
df_stadium["Team(s)"].loc[31] = "Los Angeles Chargers"

In [17]:
df_san_fran = df_san_fran.join(df_stadium.set_index("Team(s)"), on = "Home Team")

In [22]:
df_san_fran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              27 non-null     object 
 1   Home Team         27 non-null     object 
 2   Away Team         27 non-null     object 
 3   Home Score        27 non-null     int64  
 4   Away Score        27 non-null     int64  
 5   Total Score Open  27 non-null     float64
 6   Name              27 non-null     object 
 7   Location          27 non-null     object 
 8   Surface           27 non-null     object 
 9   Roof type         27 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 2.2+ KB


In [20]:
df_day_time.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, 2017-09-10 to 2020-12-31
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Time of Game  64 non-null     object
 1   Game Day      64 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


In [21]:
df_san_fran.reset_index(inplace=True)
df_san_fran["Date"] = pd.to_datetime(df_san_fran["Date"]).dt.date
#df_san_fran.set_index("Date",inplace=True)
pd.concat([df_san_fran,df_day_time])

,Date,Home Team,Away Team,Home Score,Away Score,Total Score Open,Name,Location,Surface,Roof type,Time of Game,Game Day
0,2020-09-27,New York Giants,San Francisco 49ers,9.0,36.0,42.5,MetLife Stadium,"East Rutherford, New Jersey",UBU Sports Speed Series S5-M Synthetic Turf[30],Open,NaN,NaN
1,2020-09-20,New York Jets,San Francisco 49ers,13.0,31.0,43.5,MetLife Stadium,"East Rutherford, New Jersey",UBU Sports Speed Series S5-M Synthetic Turf[30],Open,NaN,NaN
2,2020-02-02,Kansas City Chiefs,San Francisco 49ers,31.0,20.0,53.0,Arrowhead Stadium,"Kansas City, Missouri",Bermuda grass,Open,NaN,NaN
3,2019-12-29,Seattle Seahawks,San Francisco 49ers,21.0,26.0,48.0,CenturyLink Field,"Seattle, Washington",FieldTurf Revolution 360[9],Open,NaN,NaN
4,2019-12-08,New Orleans Saints,San Francisco 49ers,46.0,48.0,45.5,Mercedes-Benz Superdome,"New Orleans, Louisiana",FieldTurf Revolution 360[9],Fixed,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:00PM ET,Sun
2020-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:00PM ET,Sun
2020-12-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4:25PM ET,Sun
2020-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4:05PM ET,Sun
